In [1]:
import socket
import argparse
import json
from pprint import pprint

packet_storage = {
}


def dehex_packet(x):
    temp_packet = dict(x)
    hexed_data = ['length', 'Offset', 'ID', 'FragFlag']
    for key in temp_packet:
        if key in hexed_data:
            temp_packet[key] = unpack_hex_to_int(temp_packet[key])
    return temp_packet


def unpack_hex_to_int(data):
    return int(data, 16)

In [2]:
def reassemble(packets):
    sorted_pkt_list = sorted(packets, key=lambda k: int(k['Offset']))
    assembled_data = []
    for pkt in sorted_pkt_list:
        assembled_data.append(pkt['payload'])
    return ''.join(assembled_data)


def process_storage():
    assembled_data = []
    for key in packet_storage:
        packets = packet_storage[key]
        for pkt in packets:
            if pkt['FragFlag'] == 0 and pkt['Offset'] != 0:
                print("Received final fragment for packet ID {}".format(pkt['ID']))
                data = reassemble(packets)
                # print("Assembled data:\n {}".format(data))
                # del packet_storage[key]
                assembled_data.append({'data': data, 'id': pkt['ID']})
                break
            if pkt['FragFlag'] == 0 and pkt['Offset'] == 0:
                print("Received packet ID {}".format(pkt['ID']))
                # print("Data:\n {}".format(pkt['payload']))
                assembled_data.append({'data': pkt['payload'], 'id': pkt['ID']})
                break
    # pprint(assembled_data)
    sorted_assembled_list = sorted(assembled_data, key=lambda k: int(k['id']))
    final_data = []
    for x in sorted_assembled_list:
        final_data.append(x['data'])
    return ''.join(final_data)

In [3]:
def add_to_storage(pkt):
    packet = dict(pkt)
    global packet_storage
    packet = dehex_packet(packet)

    packet_id = str(packet['ID'])

    if packet_id in packet_storage:
        packet_storage[packet_id].append(packet)
    else:
        packet_storage[packet_id] = [packet]

In [4]:
def udp_receiver(udp_ip, udp_port):
    received_data = ''
    sock = socket.socket(socket.AF_INET,  # Internet
                         socket.SOCK_DGRAM)  # UDP
    sock.settimeout(2)
    sock.bind((udp_ip, udp_port))

    for x in range(20):
        data = ''
        try:
            data, addr = sock.recvfrom(1024)  # buffer size is 1024 bytes
        except socket.timeout:
            pass
        if data:
            received_data += data.decode()

    return received_data

In [5]:
if __name__ == '__main__':
    parser = argparse.ArgumentParser(description='Creates packet fragments and send via UDP')
    parser.add_argument('-f', '--output_filename', help='Output filename', default='received.txt')
    parser.add_argument('-ip', '--recvIP', help='receiver IP address', default='127.0.0.1')
    parser.add_argument('-port', '--recvPort', help='receiver port number', type=int, default=2001)

    args = parser.parse_args()
    print("Waiting to receive data...")
    received_data = udp_receiver(args.recvIP, args.recvPort)

    for pkt in json.loads(received_data):
        add_to_storage(pkt)

    # printing all received fragments (by ID) to file for verification purpose (AFTER UN-HEXing few fields)
    with open("recv_fragmented_packets.txt", 'w+') as myfile:
        pprint(packet_storage, myfile)

    assembled_data = process_storage()

    print("\nWriting the receive assembled data to {}".format(args.output_filename))
    with open(args.output_filename, 'w+') as myfile:
        myfile.write(json.loads(assembled_data))

Waiting to receive data...


KeyboardInterrupt: 